In [0]:
catalog = "hive_metastore"
schema = "default"
table = "customer"

In [0]:
result = spark.sql(f"DROP TABLE IF EXISTS `{catalog}`.`{schema}`.`{table}`")
display(result)

In [0]:
SQL = f"""
  CREATE TABLE IF NOT EXISTS {catalog}.{schema}.{table} (
    customer_id INT,
    first_name STRING,
    last_name STRING,
    email STRING,
    phone STRING,
    address STRING,
    city STRING,
    state STRING,
    zip_code DOUBLE,
    join_date DATE
)
USING DELTA;
"""
spark.sql(SQL)

In [0]:
SQL = f"""
INSERT INTO {catalog}.{schema}.{table}
SELECT 
    id AS customer_id,
    CASE WHEN id % 2 = 0 THEN 'John' ELSE 'Jane' END AS first_name,
    CASE WHEN id % 2 = 0 THEN 'Doe' ELSE 'Smith' END AS last_name,
    CONCAT('customer', id, '@example.com') AS email,
    CONCAT('+1-555-0', LPAD(id, 3, '0')) AS phone,
    CONCAT('123', id, ' Main St') AS address,
    CASE WHEN id % 5 = 0 THEN 'New York' 
         WHEN id % 5 = 1 THEN 'Los Angeles'
         WHEN id % 5 = 2 THEN 'Chicago'
         WHEN id % 5 = 3 THEN 'Houston'
         ELSE 'San Francisco' END AS city,
    CASE WHEN id % 5 = 0 THEN 'NY' 
         WHEN id % 5 = 1 THEN 'CA'
         WHEN id % 5 = 2 THEN 'IL'
         WHEN id % 5 = 3 THEN 'TX'
         ELSE 'CA' END AS state,
    LPAD(CAST((10000 + id) AS STRING), 5, '0') AS zip_code,
    DATE_ADD('2020-01-01', id % 365) AS join_date
FROM (
    SELECT EXPLODE(SEQUENCE(1, 100)) AS id
);
"""
spark.sql(SQL)